# 背景

AI图像抠图技术的成熟使得对准确度要求一般的视频抠图工作可以由AI算法来快速完成。本项目通过AI抠图与图像合成技术实现图片背景发生转化，同时前景中的人物的像素值保持不变。项目使用的素材文件有两个，一是人在跑步的视频，二是模拟奇幻空间的粒子飞舞视频。素材视频下载自：https://www.motionelements.com/。

本项目的学习心得链接：

Baidu Aistudio 项目开源链接：

Github 项目开源链接：


# 结果
项目运行的成果为将人物的奔跑场景更换为指定的粒子飞舞背景。考虑项目运行时间，最终生成的视频为8s时长。素材视频及最终的合成视频动图可查看学习心得链接，完整成果视频可查看[百度网盘链接](https://pan.baidu.com/s/14uMsExiAQZrW-y40fjddBw)，
提取码：42dt。

# 思路
视频处理思路为：
1. 采用OpenCV库读取视频为各帧图像；
2. 借助PaddleHub DeepLabv3+模型(deeplabv3p_xception65_humanseg)实现一键抠图，提取人像；
3. 将抠出来的人像与粒子背景根据透明度进行叠加，合成新的图片；
4. 将合成图片按照顺序制作成视频。

# 实现步骤
代码的AI抠图部分参考了paddlehub官方文档：https://www.paddlepaddle.org.cn/hubdetail?name=deeplabv3p_xception65_humanseg&en_category=ImageSegmentation。 

视频图像处理参考自：https://github.com/hamlinzheng/humanseg_video/blob/master/video_seg.py。

In [1]:
!pip install paddlehub==1.8.2 -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 337kB 10.1MB/s eta 0:00:01
  Found existing installation: paddlehub 1.6.0
    Uninstalling paddlehub-1.6.0:
      Successfully uninstalled paddlehub-1.6.0


In [2]:
# 导入包
import os
import cv2
import numpy as np
from PIL import Image
import paddlehub as hub
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg 

2020-08-30 08:42:55,910-INFO: font search path ['/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/mpl-data/fonts/ttf', '/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/mpl-data/fonts/afm', '/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/mpl-data/fonts/pdfcorefonts']
2020-08-30 08:42:56,341-INFO: generated new fontManager


In [6]:
# 路径设置
human_video_path = 'data/data51850/foreground.mp4'  # 带任务的视频路径
particle_video_path =  'data/data51850/background.mp4'  # 粒子视频路径 
human_frames_path = './human_frames/'  # 带人物的视频各帧图片存储路径
particle_frames_path = './back_frames/'  # 粒子视频各帧图片存储路径
human_seg_path = './human_seg/'  # 人像抠图的存储路径
frame_com_path = './frames_com/'  # 合成后的图片的路径
output_video_path = 'output.mp4'  # 输出视频的路径

check_exsit_list = [human_frames_path, particle_frames_path, human_seg_path, frame_com_path]
for path in check_exsit_list:
    if not os.path.exists(path):
        os.mkdir(path)

In [3]:
# 步骤一：从视频中提取各帧图像至文件夹中
def CutVideo2Image(video_path, img_path):
    cap = cv2.VideoCapture(video_path)
    index = 0
    while(True):
        ret,frame = cap.read() 
        if ret:
            cv2.imwrite(img_path + '{}.jpg'.format(index), frame)
            index += 1
        else:
            break
    cap.release()
    print('Video cut finish, all %d frame' % index)
    return index

human_frame_count = CutVideo2Image(human_video_path, human_frames_path)
particle_frame_count = CutVideo2Image(particle_video_path, particle_frames_path)
print(human_frame_count)
print(particle_frame_count)

#### 跑步视频比粒子视频要短，最终长度以跑步视频为准

In [11]:
# 步骤二：采用paddleHub的图像分割模型实现人像抠图
# 加载paddlehub的抠图模型
module = hub.Module(name="deeplabv3p_xception65_humanseg")

# 对所有人像视频进行抠图操作，保存抠图文件
human_img_path = [os.path.join(human_frames_path, fname) for fname in os.listdir(human_frames_path)]
results = module.segmentation(paths = human_img_path, use_gpu=False, visualization = True, output_dir=human_seg_path)

[2020-08-30 08:48:11,753] [    INFO] - Installing deeplabv3p_xception65_humanseg module


[==================================================] 100.00%
Uncompress /home/aistudio/.paddlehub/tmp/tmpbfi1tp16/deeplabv3p_xception65_humanseg
[==================================================] 100.00%


[2020-08-30 08:48:31,225] [    INFO] - Successfully installed deeplabv3p_xception65_humanseg-1.1.1


In [32]:
# 步骤三：将人像抠图与粒子图进行合成
for frame_id in range(human_frame_count):
    print("当前处理帧数:{}".format(frame_id))
    human_seg_frame_path = os.path.join(human_seg_path, "{}.png".format(frame_id))
    particle_frame_path = os.path.join(particle_frames_path, "{}.jpg".format(frame_id))
    combine_frame_path = os.path.join(frame_com_path, "{}.png".format(frame_id))

    fore_img = cv2.imread(human_seg_frame_path, -1)
    base_img = cv2.imread(particle_frame_path)

    fore_channels = cv2.split(fore_img)
    base_channels = cv2.split(base_img)
    b, g, r, a = cv2.split(fore_img)
    for i in range(3):
        base_channels[i][:,:] = base_channels[i][:,:]*(255.0-a)/255
        base_channels[i][:,:] += np.array(fore_channels[i]*(a/255), dtype=np.uint8)
    r = cv2.imwrite(combine_frame_path, cv2.merge(base_channels))
    

当前处理帧数:0
当前处理帧数:1
当前处理帧数:2
当前处理帧数:3
当前处理帧数:4
当前处理帧数:5
当前处理帧数:6
当前处理帧数:7
当前处理帧数:8
当前处理帧数:9
当前处理帧数:10
当前处理帧数:11
当前处理帧数:12
当前处理帧数:13
当前处理帧数:14
当前处理帧数:15
当前处理帧数:16
当前处理帧数:17
当前处理帧数:18
当前处理帧数:19
当前处理帧数:20
当前处理帧数:21
当前处理帧数:22
当前处理帧数:23
当前处理帧数:24
当前处理帧数:25
当前处理帧数:26
当前处理帧数:27
当前处理帧数:28
当前处理帧数:29
当前处理帧数:30
当前处理帧数:31
当前处理帧数:32
当前处理帧数:33
当前处理帧数:34
当前处理帧数:35
当前处理帧数:36
当前处理帧数:37
当前处理帧数:38
当前处理帧数:39
当前处理帧数:40
当前处理帧数:41
当前处理帧数:42
当前处理帧数:43
当前处理帧数:44
当前处理帧数:45
当前处理帧数:46
当前处理帧数:47
当前处理帧数:48
当前处理帧数:49
当前处理帧数:50
当前处理帧数:51
当前处理帧数:52
当前处理帧数:53
当前处理帧数:54
当前处理帧数:55
当前处理帧数:56
当前处理帧数:57
当前处理帧数:58
当前处理帧数:59
当前处理帧数:60
当前处理帧数:61
当前处理帧数:62
当前处理帧数:63
当前处理帧数:64
当前处理帧数:65
当前处理帧数:66
当前处理帧数:67
当前处理帧数:68
当前处理帧数:69
当前处理帧数:70
当前处理帧数:71
当前处理帧数:72
当前处理帧数:73
当前处理帧数:74
当前处理帧数:75
当前处理帧数:76
当前处理帧数:77
当前处理帧数:78
当前处理帧数:79
当前处理帧数:80
当前处理帧数:81
当前处理帧数:82
当前处理帧数:83
当前处理帧数:84
当前处理帧数:85
当前处理帧数:86
当前处理帧数:87
当前处理帧数:88
当前处理帧数:89
当前处理帧数:90
当前处理帧数:91
当前处理帧数:92
当前处理帧数:93
当前处理帧数:94
当前处理帧数:95
当前处理帧数:96
当前处理帧数:97
当前处理帧数:98
当前处理帧数:99
当前处理帧数:100

In [33]:
# 步骤四：将图像合成为视频
def CombVideo(in_path, out_path, size):
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(out_path,fourcc, 30.0, size)
    files = os.listdir(in_path)

    for i in range(len(files)):
        img = cv2.imread(in_path + '%d.png' % i)
        out.write(img)#保存帧
    out.release()

CombVideo(frame_com_path, output_video_path, (1920, 1080))